In [1]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import PyPDF2
import pandas as pd
from typing import List, Dict

In [4]:
#from pypdf import PdfReader

#def extract_text_pypdf(pdf_path):
#    reader = PdfReader(pdf_path)
#    text = ""
#    for page in reader.pages:
#        text += page.extract_text()
#    return text


#pdf_file = "test_data_pdf/IPI-(2012-1).pdf" 
#extracted_text = extract_text_pypdf(pdf_file)
#print(extracted_text)

In [5]:
import PyPDF2

In [6]:
def text_extraction(folder_path):
    
    data = []
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]
    
    for pdf_file in sorted(pdf_files):
        file_path = os.path.join(folder_path, pdf_file)
        text = ""
        
        try:
            with open(file_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
            
            data.append({
                'file_name': pdf_file,
                'text': text,
                'text_length': len(text)
            })
            print(f"Обработан: {pdf_file}")
            
        except Exception as e:
            print(f"Ошибка с {pdf_file}: {e}")
    
    return pd.DataFrame(data)

if __name__ == "__main__":
    folder = "test_data_pdf"
    if os.path.exists(folder):
        df = text_extraction(folder)
        if not df.empty:
            df.to_csv("journal_dataset_1.csv", index=False, encoding='utf-8')
            print("ok")

Обработан: IPI(2012-3)-full.pdf
Обработан: IPI-(2012-1).pdf
Обработан: IPI-(2012-2)full.pdf
Обработан: IPI-(2012-4).pdf
ok


In [7]:
pip install matplotlib seaborn 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Tuple
import re
import warnings
warnings.filterwarnings('ignore')

In [10]:
df = pd.read_csv("journal_dataset_1.csv")
print(df.head())

              file_name                                               text  \
0  IPI(2012-3)-full.pdf  Информатика  и её применения  \n \nТом 6 Выпус...   
1      IPI-(2012-1).pdf  Информатика  и её применения  \n \nТом 6 Выпус...   
2  IPI-(2012-2)full.pdf  Информатика  и её применения  \n \nТом 6 Выпус...   
3      IPI-(2012-4).pdf  Информатика  и её применения  \nТом 6   Выпуск...   

   text_length  
0       465685  
1       507646  
2       490996  
3       410020  


In [11]:
def preprocess_text(text: str) -> List[str]:

    if not isinstance(text, str):
        return []
    
    text = text.lower() # Нижний регистр
    
    text = re.sub(r'[^\w\s\.\,\-\+\=\*\/\(\)\[\]\{\}\^]', ' ', text) # Удаление спец символов с сохранением мат обозначений (вроде бы)
    
    text = re.sub(r'\s+', ' ', text) # Лишние проблелы
    
    # Токенизация
    tokens = text.split()
    
    # Удалить стоп-слова
    from nltk.corpus import stopwords

    stop_words = set(stopwords.words('russian') + stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2]

    return tokens


In [12]:
df['tokens'] = df['text'].apply(preprocess_text)
df['processed_text'] = df['tokens'].apply(lambda x: ' '.join(x))

In [13]:
print(sum(len(tokens) for tokens in df['tokens']))

180073


In [14]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from gensim import corpora

In [16]:
dictionary = corpora.Dictionary(df['tokens']) # Словарь
print(len(dictionary))

54434


In [17]:
# Фильтрация мловаря
dictionary.filter_extremes(no_below=2, no_above=0.8)
print(len(dictionary))

9824


In [18]:
# Корпус
corpus = [dictionary.doc2bow(tokens) for tokens in df['tokens']]
print(len(corpus)) #размер корпуса в документах (шт)

4


In [19]:
# Оценка качества

# Вычисление когерентности (насколько хорошо слова в теме связаны между собой по смыслу)
def evaluate_coherence(model, tokens: List[List[str]], dictionary, topn: int = 10) -> float:
    from gensim.models.coherencemodel import CoherenceModel
    
    if hasattr(model, 'show_topics'):
        # Для LDA, LSI
        coherence_model = CoherenceModel(
            model=model, 
            texts=tokens, 
            dictionary=dictionary, 
            coherence='c_v'
        )
    else:
        # Для других моделей
        topics = []
        for topic_id in range(model.num_topics):
            topic_words = model.get_topic_words(topic_id, topn=topn)
            topics.append([word for word, _ in topic_words])
        
        coherence_model = CoherenceModel(
            topics=topics, 
            texts=tokens, 
            dictionary=dictionary, 
            coherence='c_v'
        )
    
    return coherence_model.get_coherence()


In [20]:
def visualize_topics(model, model_name: str, num_words: int = 10):
    print(f"\n{model_name} - Темы:")
    
    if hasattr(model, 'print_topics'):
        # LDA
        topics = model.print_topics(num_words=num_words)
        for idx, topic in topics:
            print(f"Тема {idx}: {topic}")
    
    elif hasattr(model, 'show_topics'):
        # LSI
        topics = model.show_topics(num_topics=-1, num_words=num_words, formatted=True)
        for idx, topic in topics:
            print(f"Тема {idx}: {topic}")
    
    elif hasattr(model, 'get_topics'):
        # BigARTM
        for topic_idx in range(model.num_topics):
            topic_words = model.get_topic_words(topic_idx, num_words)
            words = [f"{word}({prob:.3f})" for word, prob in topic_words]
            print(f"Тема {topic_idx}: {', '.join(words)}")
    
    print()

In [21]:
# LDA
print("LDA")

from gensim.models import LdaModel

lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=5,  # Можно настроить
    random_state=42,
    passes=10,
    alpha='auto',
    per_word_topics=True
)

# Оценка
lda_coherence = evaluate_coherence(lda_model, df['tokens'], dictionary)
print(f"Когерентность: {lda_coherence:.4f}")

visualize_topics(lda_model, "LDA")

LDA
Когерентность: 0.5210

LDA - Темы:
Тема 0: 0.004*"мотивов" + 0.004*"правами" + 0.004*"учетными" + 0.003*"записями" + 0.003*"пальцев" + 0.002*"идентификации" + 0.002*"mediator" + 0.002*"nist" + 0.002*"разрешимости" + 0.002*"ладони"
Тема 1: 0.010*"random" + 0.005*"coefficients" + 0.005*"see" + 0.003*"dependent" + 0.003*"матрицы" + 0.003*"sums" + 0.003*"sequence" + 0.003*"theorem" + 0.003*"vector" + 0.003*"one"
Тема 2: 0.004*"тезауруса" + 0.003*"заявок" + 0.003*"traffic" + 0.003*"one" + 0.003*"географических" + 0.002*"random" + 0.002*"источников" + 0.002*"радиоисточников" + 0.002*"информационными" + 0.002*"электронных"
Тема 3: 0.009*"rif" + 0.004*"правилах" + 0.003*"базы" + 0.002*"языка" + 0.002*"нечетких" + 0.002*"принадлежности" + 0.002*"s(1)" + 0.002*"библиотеки" + 0.002*"пакетов" + 0.002*"библиотек"
Тема 4: 0.000*"rif" + 0.000*"random" + 0.000*"one" + 0.000*"языка" + 0.000*"средств" + 0.000*"coefficients" + 0.000*"базы" + 0.000*"theorem" + 0.000*"see" + 0.000*"мотивов"



In [23]:
#LSI
print("LSI")

from gensim.models import LsiModel

lsi_model = LsiModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=5
)

lsi_coherence = evaluate_coherence(lsi_model, df['tokens'], dictionary)
print(f"Когерентность LSI: {lsi_coherence:.4f}")

visualize_topics(lsi_model, "LSI")

LSI
Когерентность LSI: 0.5594

LSI - Темы:
Тема 0: 0.201*"rif" + 0.182*"random" + 0.101*"one" + 0.094*"мотивов" + 0.094*"правами" + 0.090*"учетными" + 0.087*"записями" + 0.087*"базы" + 0.081*"правилах" + 0.079*"средств"
Тема 1: 0.342*"random" + -0.314*"rif" + 0.170*"see" + 0.158*"coefficients" + -0.125*"правилах" + 0.113*"one" + 0.107*"dependent" + -0.102*"базы" + 0.100*"theorem" + 0.096*"sums"
Тема 2: 0.215*"rif" + -0.190*"мотивов" + -0.185*"правами" + -0.180*"учетными" + -0.168*"записями" + -0.139*"пальцев" + -0.113*"mediator" + -0.113*"идентификации" + -0.100*"nist" + -0.098*"ладони"
Тема 3: 0.218*"rif" + -0.203*"тезауруса" + 0.160*"random" + -0.159*"заявок" + -0.148*"traffic" + 0.144*"coefficients" + -0.124*"географических" + 0.115*"see" + -0.101*"радиоисточников" + -0.099*"информационными"

